# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def habr_search(queries):
  df = []
  for query in queries:
    url = f"https://habr.com/ru/search/?q={query}"
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, "html.parser")
      articles = soup.find_all("article", class_="tm-articles-list__item")
      for article in articles:
        date_element = article.find("time")
        date = date_element.get("title") if date_element else None

        title_element = article.find("h2", class_="tm-title")
        title = title_element.text.strip() if title_element else None

        link_element = article.find("a", class_="tm-title__link")
        link = link_element.get("href") if link_element else None
        full_link = f"https://habr.com{link}" if link else None

        if date and title and full_link:
          df.append(pd.DataFrame({"date": [date], "title": [title], "link": [full_link]}))
    else:
      print(f"Ошибка при поиске по запросу {query}: {response.status_code}")

  result_df = pd.concat(df, ignore_index=True)
  result_df = result_df.drop_duplicates(subset=["date", "title", "link"])

  return result_df


In [3]:

queries = ["python", "анализ данных"]
df = habr_search(queries)
df


,date,title,link
0,"2024-02-02, 11:30",Ресурсы для поиска работы Python разработчикам,https://habr.com/ru/articles/790916/
1,"2023-10-09, 06:46",10 лучших практик логирования в Python,https://habr.com/ru/companies/ruvds/articles/7...
2,"2024-02-01, 00:29",Настройка C++ проекта c OpenMP. Обертывание С+...,https://habr.com/ru/articles/790538/
3,"2024-02-01, 14:01",Ускоряем анализ данных в 170 000 раз с помощью...,https://habr.com/ru/companies/ncloudtech/artic...
4,"2023-04-29, 10:09",7 и 1/2 подходов к проверке атрибутов классов ...,https://habr.com/ru/articles/732388/
5,"2023-12-14, 13:54",Как написать проект по автоматизации на Python...,https://habr.com/ru/companies/uchi_ru/articles...
6,"2023-10-03, 03:21",Год ожиданий — и мы получили Python 3.12. Изме...,https://habr.com/ru/companies/selectel/article...
7,"2024-01-30, 12:25",Продюсеры и консьюмеры с Apache Kafka в Python,https://habr.com/ru/companies/otus/articles/78...
8,"2023-10-16, 16:54",Примитивы синхронизации в Python Asyncio: Исче...,https://habr.com/ru/articles/767792/
9,"2020-04-21, 18:35","Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/498364/
